In [4]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
@author: pho
"""
import sys
from threading import Thread
from ipygany import PolyMesh, Scene, IsoColor, WarpByScalar
import pyvista as pv
import pyvistaqt as pvqt
import colorcet as cc # Colormaps:

# pv.set_jupyter_backend('pythreejs')
# pv.set_jupyter_backend('ipygany') # ipygany fails " Failed to use notebook backend: "
# pv.set_jupyter_backend('panel') # Doesn't work either, produces no output
import numpy as np
import h5py
import hdf5storage # conda install hdf5storage
from pathlib import Path
import bqplot.scales
import seaborn as sns
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.colors import Normalize
# import mplcursors
import math # For color map generation
from matplotlib.colors import ListedColormap
from matplotlib.cm import hsv

import ipywidgets as widgets
# Imports PIL module
# import PIL # For applying the binary mask as a image texture to a mesh
# import opencv
# from IPython.display import display # For manual display customization
# from icecream import ic # For debugging
# def icecream_debugPrint(obj):
#     if isinstance(obj, list):
#          return '<list ({})>: {}'.format(len(obj), obj)
#     elif isinstance(obj, np.ndarray):
#         return '<np.array ({})>: {}'.format(np.shape(obj), obj)
#     return repr(obj)

# ic.configureOutput(argToStringFunction=icecream_debugPrint)
# from PyQt5 import QtWidgets, uic
from pyvistaqt import QtInteractor, MainWindow

In [5]:
# import PhoPositionalData as pdp
# from PhoPositionalData import load_exported, process_data
from PhoPositionalData.load_exported import *
# from PhoPositionalData.process_data import process_positionalAnalysis_data, gen_2d_histrogram, get_heatmap_color_vectors, process_chunk_equal_poritions_data, extract_spike_timeseries
from PhoPositionalData.process_data import *
from PhoPositionalData.plot_data import *
from PhoPositionalData.import_data import *

In [7]:
# RoyMaze1:
# mat_import_parent_path = Path(r'C:\Share\data\RoyMaze1')
# '/Users/pho/repo/Python Projects/PhoNeuronGillespie2021CodeRepo/PhoMatlabDataScripting/ExportedData/RoyMaze1/'

# mat_import_parent_path = Path(r'/Users/pho/repo/Python Projects/PhoNeuronGillespie2021CodeRepo/PhoMatlabDataScripting/ExportedData/RoyMaze1')
# mat_import_parent_path = Path(r'data/ExportedData/RoyMaze1')
mat_import_parent_path = Path(r'data\ExportedData\KevinMaze1')

# mat_import_file = r'C:\Share\data\RoyMaze1\ExportedData.mat'
# Import the positions
t,x,y,speeds,dt,dx,dy = perform_import_positions(mat_import_parent_path=mat_import_parent_path)
# Import the spikes
spike_matrix, spike_cells, num_cells, spike_list, spike_positions_list, cell_ids, reverse_cellID_idx_lookup_map = perform_import_spikes(t, x, y, mat_import_parent_path=mat_import_parent_path)
print('num_cells: {}'.format(num_cells))
print('cell_ids: {}'.format(cell_ids))

Loading matlab import file: data\ExportedData\KevinMaze1\ExportedData\positionAnalysis.mat...


OSError: Reader needs file name or open file-like object

In [ ]:
# reverse_cellID_idx_lookup_map[2]
active_cells_colormap = cc.rainbow[:num_cells]
# active_cells_colormap

In [ ]:
# # Load required behavioral data:
# # all_results_mat_import_file = mat_import_parent_path.joinpath('PhoResults_Expt1_RoyMaze1.mat') # All data, very slow to load
# # all_results_data = import_mat_file(mat_import_file=all_results_mat_import_file)
# # behavioral_periods = all_results_data['active_processing/behavioral_periods_table']
# # ripple_data = all_results_data['active_processing/source_data/ripple/RoyMaze1']
# # ripple_times = ripple_data['time']
# # ripple_peak_times = ripple_data['peakTime']
# # ripple_peak_values = ripple_data['peakVale']

# ripple_times, ripple_peak_times, ripple_peak_values, behavioral_periods, behavioral_epochs = perform_import_extras(mat_import_parent_path=mat_import_parent_path)

# # behavioral_periods = all_results_data['active_processing/behavioral_periods_table']
# # print('spike_matrix: {}, spike_cells: {}'.format(np.shape(spike_matrix), np.shape(spike_cells)))
# num_periods = np.shape(behavioral_periods)[0]
# num_ripples = np.shape(ripple_times)[0]
# print('num_periods: {}'.format(num_periods))

In [ ]:
# print(behavioral_periods)
# np.shape(behavioral_periods)

In [ ]:
# print(ripple_times)

In [ ]:
# # Filter the spikes by ripple_times
# # ripple_times[0]
# # [for i in num_ripples]


# # for i in np.arange(num_cells)
# # i = 0

# # np.true_like(spike_cells[i])
# # for j in np.arange(num_ripples):
# j = 99 # For a specific ripple:
# foundItems = [] # Empty list
# for i in np.arange(num_cells):
#     foundItems.append(np.where(np.logical_and(spike_cells[i]>=ripple_times[j,0], spike_cells[i]<=ripple_times[j,1])))
#     # foundItems = np.where(np.logical_and(spike_cells[i]>=ripple_times[j,0], spike_cells[i]<=ripple_times[j,1]))

# print(foundItems)
        
# # print(foundItems)

In [ ]:
# Plot the spikes on the track:
# min_timestep = 0
# max_timestep = 40
min_timestep = min(t)
max_timestep = max(t)
print('min_timestep: {}. max_timestep: {}'.format(min_timestep, max_timestep))
active_spike_indices, active_spike_list, active_spike_positions_list = get_filtered_window(spike_list, spike_positions_list, min_timestep=0, max_timestep=max_timestep)
# ic(active_spike_list, active_spike_positions_list)

def build_flat_map_plot_data():
    # Builds the flat base maze map that the other data will be plot on top of
    z = np.zeros_like(x)
    # ic(x, y, z)
    point_cloud = np.vstack((x, y, z)).T
    # ic(point_cloud)
    pdata = pv.PolyData(point_cloud)
    pdata['occupancy heatmap'] = np.arange(np.shape(point_cloud)[0])

    # create many spheres from the point cloud
    # geo = pv.Sphere(radius=0.2, phi_resolution=3, theta_resolution=3)
    # geo = pv.ParametricSuperEllipsoid(n1=0.1, n2=2)
    # geo = pv.ParametricSuperEllipsoid(xradius=0.2, yradius=0.2, zradius=0.02, n1=0.1, n2=0.1) # makes a square title geometry
    geo = pv.Circle(radius=0.5)
    pc = pdata.glyph(scale=False, geom=geo)
    return pdata, pc

    
def build_cell_spikes_plot_data(cell_index):
    spike_series = active_spike_positions_list[cell_index]
    spike_series_times = active_spike_list[cell_index]
    spike_series_positions = active_spike_positions_list[cell_index]
    # ic(spike_series_positions)
    # ic(type(spike_series_positions))
    # ic(spike_series_positions[0,:])
    # print('spike_series_positions: {}'.format(spike_series_positions))
    # print('cell_id: {}'.format(cell_id))
    # print(np.shape(spike_series_times))
    z = np.zeros_like(spike_series_positions[0,:])
    point_cloud = np.vstack((spike_series_positions[0,:], spike_series_positions[1,:], z)).T
    # ic(point_cloud)
    pdata = pv.PolyData(point_cloud)
    # pdata['occupancy heatmap'] = np.arange(np.shape(point_cloud)[0])
#     pdata['occupancy heatmap'] = spike_series_times
    pdata['cellID'] = (np.ones_like(spike_series_positions[0,:]) * cell_index)
    # create many spheres from the point cloud
    # spike_geom = pv.Sphere(radius=0.2, phi_resolution=3, theta_resolution=3)
    # spike_geom = pv.Arrow(direction=(0.0, 0.0, 1.0), shaft_resolution=8, scale=3)
    spike_geom = pv.Cone(direction=(0.0, 0.0, -1.0), height=10.0, radius=0.2)
    pc = pdata.glyph(scale=False, geom=spike_geom)
#     pc['cellID'] = pdata['cellID']
    # # create many spheres from the point cloud
    # # sphere = pv.Sphere(radius=2.0, phi_resolution=3, theta_resolution=3)
    # spike_geom = pv.Arrow()
    # pc = pdata.glyph(scale=False, factor=0.0003, geom=spike_geom)
    return pdata, pc

# def initialize_plot():
#     p.add_bounding_box()
#     p.show_grid()
#     p.add_axes(line_width=5, labels_off=False)

    
def complete_plot_for_cell_idx(value):
    # pass in the original_cell_id you want to display the plot for:
    original_cell_id = int(value)
    cell_index = reverse_cellID_idx_lookup_map[original_cell_id]
    pdata, pc = build_cell_spikes_plot_data(cell_index)
    
    # Plot the arrows:
    # Add scalar array with range (0, 100) that correlates with elevation
#     mesh['values'] = pv.plotting.normalize(mesh['Elevation']) * 100

#     p.add_mesh(pc, name='spikes_main', color="green")
    p.add_mesh(pc, name='spikes_main', scalars='cellID', color=active_cells_colormap[original_cell_id])
    p.add_text('Spikes for unit_id: {}'.format(original_cell_id), name='txtActive_index', position='upper_edge')
    # p.show()

    

    
# Initial plot setup:
p = pvqt.BackgroundPlotter() # Use just like you would a pv.Plotter() instance
# p = pv.Plotter() # Need a non-background plotter instance to work with add_slider_widget apparently
p.clear()


# reverse lookup cell ids:L
# cell_index = 31 # good in the middle
# cell_index = 93 # biased toward early end
# cell_index = 95 # 
cell_index = 90
cell_index = 31

# complete_plot_for_cell_idx(cell_index)
    
slider = p.add_slider_widget(
     complete_plot_for_cell_idx,
     [0, num_cells-1],
     title="Unit Idx",
     title_opacity=0.5,
     title_color="red",
     fmt='%0.0f',
     title_height=0.08,
)


# cell_index = 64
# pdata, pc = build_cell_spikes_plot_data(cell_index)
# pdata_maze, pc_maze = build_flat_map_plot_data()

# # Plot the flat arena
# # p.add_points(pdata)
# p.add_mesh(pc_maze, color="black")

# # Plot the arrows:
# p.add_mesh(pc, color="green")

# # cell_index = 1
# # pdata, pc = build_cell_plot_data(cell_index)
# # p.add_mesh(pc)

# # cell_index = 52
# # pdata, pc = build_cell_plot_data(cell_index)
# # p.add_mesh(pc)

pdata_maze, pc_maze = build_flat_map_plot_data()
# Plot the flat arena
# p.add_points(pdata)
p.add_mesh(pc_maze, name='maze_bg', color="black")
    
p.add_bounding_box()
p.show_grid()
p.add_axes(line_width=5, labels_off=False)
    
p.show()

In [ ]:
# pc.plot(cmap='Reds', notebook=True, jupyter_backend='ipygany') # ipygany fails " Failed to use notebook backend: "

In [ ]:
pc.plot(cmap='Reds', notebook=True, jupyter_backend='panel')

In [ ]:
# Projected Maze 2D Outline:
maze_outline_x = x
maze_outline_y = y


In [ ]:
# Split the position data into equal sized chunks to be displayed at a single time. These will look like portions of the trajectory and be used to animate. # Chunk the data to create the animation.
# curr_view_window_length = 150 # View 5 seconds at a time (30fps)
curr_view_window_length = 30 # View 5 seconds at a time (30fps)
# The original length 324574 / 30 = 10819
t_fixedSegements,x_fixedSegements,y_fixedSegements,speeds_fixedSegements,dt_fixedSegements,dx_fixedSegements,dy_fixedSegements = process_chunk_equal_poritions_data(t, x, y, speeds, dt, dx, dy, curr_view_window_length)
print('shapes - t_fixedSegements: {}, x_fixedSegements: {}, y_fixedSegements: {}'.format(np.shape(t_fixedSegements), np.shape(x_fixedSegements), np.shape(y_fixedSegements)))
z_fixedSegements = np.zeros_like(x_fixedSegements)


In [ ]:
num_time_points = np.shape(x_fixedSegements)[0]
print('num_time_points: {}\n'.format(num_time_points))
animal_location_sphere = pv.Sphere(radius=2.3)
animal_location_direction_cone = pv.Cone()

#     animal_direction_arrow = pv.Arrow()
last_only_opacity_values = np.zeros([curr_view_window_length,])
last_only_opacity_values[-1] = 1.0

# last_only_opacity_values = np.zeros([1,curr_view_window_length])
# last_only_opacity_values[-1] = 1.0
gradually_fading_opacity_values = np.arange(curr_view_window_length)

# active_trail_opacity_values = last_only_opacity_values.copy()
active_trail_opacity_values = gradually_fading_opacity_values.copy()
# print('active_trail_opacity_values: {}\n'.format(np.shape(active_trail_opacity_values)))
    
## Slider with Callback Function Example:

# p = pvqt.BackgroundPlotter() # Use just like you would a pv.Plotter() instance

def on_slider_update_mesh(value):
    curr_i = int(value)
    # point_cloud_fixedSegements = np.vstack((x_fixedSegements, y_fixedSegements, z_fixedSegements)).T
    point_cloud_fixedSegements = np.column_stack((x_fixedSegements[curr_i,:], y_fixedSegements[curr_i,:], z_fixedSegements[curr_i,:]))
#     print('point_cloud_fixedSegements: {}\n'.format(np.shape(point_cloud_fixedSegements)))
    curr_animal_point = point_cloud_fixedSegements[-1,:] # Get the last point
#     actor = p.add_mesh(pyvista.Cube(), show_edges=True)
#     mesh = pv.PolyData(point_cloud_fixedSegements)
#     mesh.plot(point_size=10, style='points')
    pdata = pv.PolyData(point_cloud_fixedSegements) # a mesh
#     pdata['orig_sphere'] = np.arange(np.shape(point_cloud)[0])
    
#     print(last_only_opacity_values)
    
#     pdata.point_data['pho_fade_values'] = np.arange(np.shape(point_cloud_fixedSegements)[0])
    pdata.point_data['pho_fade_values'] = active_trail_opacity_values

    # create many spheres from the point cloud
    pc = pdata.glyph(scale=False, geom=animal_location_sphere)
    # pc.point_data['pho_fade_values'] = active_trail_opacity_values
        
    # plines = p.add_lines(point_cloud_fixedSegements, name='animal_trajectory_lines', color='orange')
    # plines.point_data['pho_fade_values'] = active_trail_opacity_values
    #     p.add_mesh(pc, name='animal_location', color="red", opacity='linear', scalars='pho_fade_values', nan_opacity=0.0)
    # p.add_mesh(pc, name='animal_location', color='red', ambient=0.6, opacity=0.8, nan_opacity=0.0)
    # p.add_mesh(pc, name='animal_location', color='red', ambient=0.6, opacity="pho_fade_values", nan_opacity=0.0)

    p.add_mesh(pc, name='animal_location', color='red', ambient=0.6, opacity=0.8, nan_opacity=0.0, render_lines_as_tubes=True)
#     p.add_mesh(animal_location_sphere, color="grey", ambient=0.6, opacity=0.5, show_edges=False)
    
#     # Create spline with 1000 interpolation points
#     spline = pv.Spline(point_cloud_fixedSegements)
# #     # add scalars to spline and plot it
#     spline["scalars"] = np.arange(spline.n_points)
#     tube = spline.tube(radius=0.1)
# #     tube.plot(smooth_shading=True)
#     p.add_mesh(tube, name='tube')

#     sphere = pv.Sphere(phi_resolution=res, theta_resolution=res)
#     p.add_mesh(sphere, name='sphere', show_edges=True)
    return


        
p = pvqt.BackgroundPlotter() # Use just like you would a pv.Plotter() instance
# p = pv.Plotter() # Use just like you would a pv.Plotter() instance
# p.background_color = 'black'
p.add_points(pdata)
# p.add_slider_widget(on_slider_update_mesh, [0, (num_time_points-1)], title='Trajectory Timestep', event_type='always', style='modern')
# active_frame_range = np.arange(num_time_points) # All frames
active_frame_range = np.arange(num_time_points) # All frames
# active_frame_range = np.arange(100) # Only 100 frames
# active_frame_range = range(100) # Only 100 frames                  
# make_mp4_from_plotter(p, active_frame_range, on_slider_update_mesh, filename='complete_plotting.mp4')
# p.add_bounding_box()
# p.show_grid()
# p.add_axes(line_width=5, labels_off=True)
# p.hide_axes()
# p.show()
                  
print('all done!')

In [ ]:
## Qt-based GUI - Plots a window with a controllable slider with two handles at the bottom
class PyvistaQtGuiMainWindow(MainWindow):

    def __init__(self, parent=None, show=True):
        QtWidgets.QMainWindow.__init__(self, parent)

        # create the frame
        self.frame = QtWidgets.QFrame()
        vlayout = QtWidgets.QVBoxLayout()

        # add the pyvista interactor object
        self.plotter = QtInteractor(self.frame)
        vlayout.addWidget(self.plotter.interactor)
        self.signal_close.connect(self.plotter.close)

        self.rs = QRangeSlider()
        self.rs.show()
        self.rs.setRange(15, 35)
        self.rs.setBackgroundStyle('background: qlineargradient(x1:0, y1:0, x2:0, y2:1, stop:0 #222, stop:1 #333);')
        self.rs.handle.setStyleSheet('background: qlineargradient(x1:0, y1:0, x2:0, y2:1, stop:0 #282, stop:1 #393);')
        vlayout.addWidget(self.rs)
    
        self.frame.setLayout(vlayout)
        self.setCentralWidget(self.frame)

        # simple menu to demo functions
        mainMenu = self.menuBar()
        fileMenu = mainMenu.addMenu('File')
        exitButton = QtWidgets.QAction('Exit', self)
        exitButton.setShortcut('Ctrl+Q')
        exitButton.triggered.connect(self.close)
        fileMenu.addAction(exitButton)

        # allow adding a sphere
        meshMenu = mainMenu.addMenu('Mesh')
        self.add_sphere_action = QtWidgets.QAction('Add Sphere', self)
        self.add_sphere_action.triggered.connect(self.add_sphere)
        meshMenu.addAction(self.add_sphere_action)

        if show:
            self.show()

    def add_sphere(self):
        """ add a sphere to the pyqt frame """
        sphere = pv.Sphere()
        self.plotter.add_mesh(sphere, show_edges=True)
        self.plotter.reset_camera()


# Actually show the window:
if __name__ == '__main__':
    app = QtWidgets.QApplication(sys.argv)
    window = PyvistaQtGuiMainWindow()
    app.exec_()
    
